In [12]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join('../..')))
from config import db_password

import psycopg2
from sqlalchemy import create_engine
import urllib.parse

db_string = f"postgresql://postgres:{urllib.parse.quote(db_password)}\
@127.0.0.1:5432/disease_prediction"

sql_dfs = {
    'dataset_bool': 'dataset_bool.csv',
    'disease_cases': 'dataset_clean.csv',
    'disease_description': 'disease_description_clean.csv',
    'disease_precautions': 'disease_precaution_clean.csv',
    'symptom_severity': 'symptom_severity_clean.csv'
}

join_query = "CREATE TABLE disease_info AS  \
SELECT * FROM disease_description AS d_desc \
INNER JOIN disease_precautions AS d_pre     \
USING (\"Disease\")"

with create_engine(db_string).connect() as engine:
    for tablename in sql_dfs:
        filename = sql_dfs[tablename]
        df = pd.read_csv(f'./Data/Cleaned/{filename}')
        try:
            df.to_sql(name=tablename,
            con=engine,
            if_exists='fail',
            index=False,
            )
        except ValueError:
            continue

    engine.execute(join_query)